# **Hackathon 2 Team 1 Project - Health Insurance Analysis**

# Section 1

## Objectives

In this section, the aim is to prepare a cleaned dataset for visualization and analysis from the raw data files. There are ETL procedures form data extraction, data cleaning and processing to data load.


## Inputs

* Datasets used for this analysis is the retail data set from Kaggle (https://www.kaggle.com/datasets/manjeetsingh/retaildataset). 

* 1 raw file will be used.
    *[insurance.csv](../dataset/raw/insurance.csv) 

## Outputs

* A cleaned dataset will be save as a CSV file below
    *  [insurance_cleaned.orig.csv](../dataset/processed/insurance_cleaned_orig.csv)
    * [insurance_cleaned_transformed.csv](../dataset/processed/insurance_cleaned_transformed.csv)





---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/Users/denniskwok/Documents/data-analytics/Hackathon2Team1Project-Health-Insurance-Analysis/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/Users/denniskwok/Documents/data-analytics/Hackathon2Team1Project-Health-Insurance-Analysis'

---

# Part A

# Data Extraction

In [4]:
import numpy as np
import pandas as pd

* **Step 1: Load Dataset**

In [5]:
# Load dataset from csv file
def load_csv(filepath):
    try:
        df = pd.read_csv(filepath)
        print(f"Loaded {filepath} successfully.")
        return df
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return pd.DataFrame()

df_insurance = load_csv("dataset/raw/insurance.csv")


Loaded dataset/raw/insurance.csv successfully.


* **Step 2: Overview The RAW Dataset**

**General dataframe information** 

In [6]:
df_insurance.info() # Display dataframe information  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


**Dataframe data overview**

In [7]:
df_insurance.head() # Display the first few rows of the dataframe

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


**Checking missing values**

In [8]:
df_insurance.isnull().sum() # Check for missing values

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

---

# Part B

# Data Transformation (data cleaning and processing)

* **Step 1: Create ID Column before Transformation**

    * Prepare for potential merge and connection between different datasets.

In [9]:
# Add ID column
df_insurance['ID'] = range(1, len(df_insurance) + 1)

# Move ID to first column
cols = ['ID'] + [col for col in df_insurance.columns if col != 'ID']
df_insurance = df_insurance[cols]

Create Categorical Column for Obesity

In [10]:
# Create Categorical Column for Obesity
def categorize_bmi(bmi):
    if bmi < 18.5:
        return 'underweight'
    elif 18.5 <= bmi < 24.9:
        return 'normal'
    elif 25 <= bmi < 29.9:
        return 'overweight'
    else:
        return 'obese'  
    
df_insurance['bmi_category'] = df_insurance['bmi'].apply(categorize_bmi)   


* **Step 2: Create Pipeline for Transformation**

     * i. Drop Missing Data
     * ii. Convert Categorical Columns to Numerical
     * iii. Scale the Continuous Numerical Columns

In [11]:
from feature_engine.encoding import OneHotEncoder
from feature_engine.imputation import DropMissingData
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('drop_missing', DropMissingData()),
    ('onehot_encoder', OneHotEncoder(variables=['sex', 'smoker', 'region', 'bmi_category'], drop_last=True))
])

df_insurance_transformed = pipeline.fit_transform(df_insurance)

**Check the OneHotEncoder Mapping**

In [12]:
# Access the fitted OneHotEncoder from the pipeline
encoder_step = pipeline.named_steps['onehot_encoder']


# Show the mapping dictionary
print('Mapping Dictionary:\n')
print(encoder_step.encoder_dict_, '\n\n')

print('New Columns after One-Hot Encoding:\n')

#new columns after the full pipeline transformation
print(df_insurance_transformed.columns.tolist(), '\n\n')

Mapping Dictionary:

{'sex': ['female'], 'smoker': ['yes'], 'region': ['southwest', 'southeast', 'northwest'], 'bmi_category': ['overweight', 'obese', 'normal']} 


New Columns after One-Hot Encoding:

['ID', 'age', 'bmi', 'children', 'charges', 'sex_female', 'smoker_yes', 'region_southwest', 'region_southeast', 'region_northwest', 'bmi_category_overweight', 'bmi_category_obese', 'bmi_category_normal'] 




* **Step 3: Make a Log Transformation for Charges**

In [13]:
import numpy as np

# Create a new column with log-transformed charges
df_insurance_transformed["log_charges"] = np.log(df_insurance_transformed["charges"])

* **Step 4: Final Check for Missing Values after Feature-engine Transformation**

In [14]:
# Check missing values in the insurance DataFrame after transformation
df_insurance_transformed.isnull().sum()

ID                         0
age                        0
bmi                        0
children                   0
charges                    0
sex_female                 0
smoker_yes                 0
region_southwest           0
region_southeast           0
region_northwest           0
bmi_category_overweight    0
bmi_category_obese         0
bmi_category_normal        0
log_charges                0
dtype: int64

There is no missing data in the dataset.

* **Step 5: Overview and Inspect the data after Feature-Engineering Transformation**

In [15]:
# Overview head and info for final transformed dataframe
display(df_insurance_transformed.head())
print(df_insurance_transformed.columns.tolist())
df_insurance_transformed.info()

,ID,age,bmi,children,charges,sex_female,smoker_yes,region_southwest,region_southeast,region_northwest,bmi_category_overweight,bmi_category_obese,bmi_category_normal,log_charges
0,1,19,27.900,0,16884.92400,1,1,1,0,0,1,0,0,9.734176
1,2,18,33.770,1,1725.55230,0,0,0,1,0,0,1,0,7.453302
2,3,28,33.000,3,4449.46200,0,0,0,1,0,0,1,0,8.400538
3,4,33,22.705,0,21984.47061,0,0,0,0,1,0,0,1,9.998092
4,5,32,28.880,0,3866.85520,0,0,0,0,1,1,0,0,8.260197


['ID', 'age', 'bmi', 'children', 'charges', 'sex_female', 'smoker_yes', 'region_southwest', 'region_southeast', 'region_northwest', 'bmi_category_overweight', 'bmi_category_obese', 'bmi_category_normal', 'log_charges']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       1338 non-null   int64  
 1   age                      1338 non-null   int64  
 2   bmi                      1338 non-null   float64
 3   children                 1338 non-null   int64  
 4   charges                  1338 non-null   float64
 5   sex_female               1338 non-null   int64  
 6   smoker_yes               1338 non-null   int64  
 7   region_southwest         1338 non-null   int64  
 8   region_southeast         1338 non-null   int64  
 9   region_northwest         1338 non-null   int64  
 10  bmi_category_overweig

---

# Part C

# Data Load (creating a cleaned dataset)

After review and processing, 2 datasets had been prepared. The original dataset Insurance_clearned_orig.csv is mainly for general visualization while the transformed dataset Insurance_cleaned_transformed is prepared for further Regression and Predictive model application.

In [16]:
# Export the cleaned dataframe to a CSV file in the folder for processed CSV
df_cleaned_orig = df_insurance.copy()
df_cleaned_orig.to_csv('dataset/processed/Insurance_cleaned_orig.csv', index=False)
df_cleaned_transformed = df_insurance_transformed.copy()
df_cleaned_transformed.to_csv('dataset/processed/Insurance_cleaned_transformed.csv', index=False)

print(f'Two cleaned dataset exported to dataset/processed/')

Two cleaned dataset exported to dataset/processed/


---

To be contined in Section 2 Data Visualization.